Basic Imports

In [3]:
import pandas
import marketData
import Orders
import Plot
import Indicators

Adding indicators

In [4]:
#df = marketData.getPrices("BTCUSDT", "1m", 78)
df = pandas.read_csv("Data/BTCUSDT1m.csv")
df = Indicators.Stochastic(df)
df = Indicators.ADX(df)
df = df.iloc[27:]
df.to_csv("Data/BTCUSDT1mQ.csv")
print("Done!")


FileNotFoundError: [Errno 2] No such file or directory: 'Data/BTCUSDT1m.csv'

Getting order book

In [ ]:
df = Orders.OrderBook("ADAUSDT")
Plot.plotSR(df[0], df[1])

Getting historical data and computing indicators

In [ ]:
import time

#Time to nanoseconds
pair = "BTCUSDT"
timeframe = "1m"
current = time.time() * 1000
current = round(current)

#Time step for requests
block = 500*60*1000
current -= block
amount = 15

#Populating latest
df = marketData.getPrices(pair, timeframe, 500)
for i in range(0, amount):
    print(str((i+1 / amount)*100)+"%")
    df = df.append(marketData.getHistorical(pair, timeframe, current - block, current))
    current -= block

df = df.sort_index()
df.to_csv("Data/"+pair+timeframe+".csv")

print("Done fetching")


df = pandas.read_csv("Data/"+pair+timeframe+".csv")
df = Indicators.Stochastic(df)
df = Indicators.ADX(df)
df = df.iloc[27:]
df.to_csv("Data/"+pair+timeframe+"Q.csv")

print("Done adding indicators")

Create training dataset

In [ ]:
import pandas as pd
features = pd.read_csv("Data/values.csv")
labels = pd.read_csv("Data/labels.csv")
labels.columns=["Position"]

df = pd.concat([features,labels], axis=1)
print(df)
df.to_csv("Data/validationdataset.csv")
print("Done!")

Bot testing

In [1]:
import bot
import time

bot = bot.bot("BTCUSDT", "model")
bot.start()
time.sleep(60)
bot.stop()

Bot started
Thread Started
2021-04-04 16:55:00.705200
Neutral: 0.9345996975898743 Long: 0.008372841402888298 Short: 0.05702745169401169


testing bridge

In [ ]:
import bridge
import marketData


df = marketData.getPrices("BTCUSDT", "1m", 1)
lastprice = df["Close"][0]

amount = 200/lastprice.item() # Dollar amount * Leverage

bridge = bridge.bridge("http://testnet.binancefuture.com", "BTCUSDT")
bridge.takePosition("SELL", lastprice * 0.9985, lastprice * 1.001, amount)

order = bridge.orders[0]
response = bridge.orderStatus(order)
print(response)
